In [22]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
#import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [23]:
#model_path = "gpt2-small/checkpoint-pretrainedtokenizer_10M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_10M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt2-small/checkpoint-pretrainedtokenizer_100M"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M"
model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [9]:
print("Special tokens:", tokenizer.all_special_tokens)
print("Special token IDs:", tokenizer.all_special_ids)
print("Special tokens map:", tokenizer.special_tokens_map)

Special tokens: ['<|endoftext|>']
Special token IDs: [50257]
Special tokens map: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


In [12]:
surprisals

[[('<pad>', 0.0),
  ('ĠTheĠ', 6.771759033203125),
  ('balloon', 12.995538711547852),
  ('Ġwa', 3.036536693572998),
  ('s', 0.0033515978138893843),
  ('Ġinf', 14.873947143554688),
  ('lat', 3.8473756313323975),
  ('ingĠfor', 9.053845405578613),
  ('Ġ10', 10.562509536743164),
  ('Ġminute', 3.032411813735962),
  ('s', 0.0010208890307694674)]]

In [24]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    print(token_surprisals)


    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0), ('know', 4.365269184112549), ('that', 3.4667656421661377), ('your', 9.112505912780762), ('friend', 6.371501922607422), ('gave', 10.674038887023926), ('a', 4.927915096282959), ('bag', 9.063486099243164), ('u', 15.337505340576172), ('ette', 13.791769981384277), ('to', 5.820535182952881), ('Mary', 13.028478622436523), ('last', 11.46209716796875), ('weekend', 7.374884128570557), ('.', 2.0132205486297607)]


[('I', 0.0),
 ('know', 4.365269184112549),
 ('that', 3.4667656421661377),
 ('your', 9.112505912780762),
 ('friend', 6.371501922607422),
 ('gave', 10.674038887023926),
 ('a', 4.927915096282959),
 ('baguette', 38.19276142120361),
 ('to', 5.820535182952881),
 ('Mary', 13.028478622436523),
 ('last', 11.46209716796875),
 ('weekend', 7.374884128570557),
 ('.', 2.0132205486297607)]

In [44]:
def calculate_sentence_surprisal(word_surprisals):
    """Returns total surprisal and average surprisal per word."""
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg


def compute_wh_licensing_interaction(sentences):
    """
    sentences: dict with keys
        'fg' = +Filler, +Gap
        'fng' = +Filler, −Gap
        'nfg' = −Filler, +Gap
        'nfng' = −Filler, −Gap
    Each value is a sentence string.

    Returns:
        A dict with total surprisal per sentence,
        average surprisals per sentence,
        and the wh-licensing interaction score.
    """
    scores = {}
    for key, sentence in sentences.items():
        word_surprisals = calculate_surprisal(sentence)
        total, avg = calculate_sentence_surprisal(word_surprisals)
        scores[key] = {'total': total, 'avg': avg, 'details': word_surprisals}

    # compute wh-licensing interaction
    interaction = (
        (scores['fng']['total'] - scores['nfng']['total']) -
        (scores['fg']['total'] - scores['nfg']['total'])
    )

    return {
        'scores': scores,
        'interaction': interaction
    }


#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [8]:
sentence_df = pd.read_csv('test_sentences/Double Gap Construction.csv')
sentence_df

,item_id,condition,filler,gap,subject_gap,object_gap,prefix,licensor,subj,verb,object,modifier
0,1,a,0,0,0,0,James realized,that,the dog,chased,the cat,through the yard.
1,1,b,0,1,1,0,James realized,that,NaN,chased,the cat,through the yard.
2,1,c,0,1,0,1,James realized,that,the dog,chased,NaN,through the yard.
3,1,d,0,1,1,1,James realized,that,NaN,chased,NaN,through the yard.
4,1,e,1,0,0,0,James realized,what,the dog,chased,the cat,through the yard.
...,...,...,...,...,...,...,...,...,...,...,...,...
155,20,d,0,1,1,1,The principal knows,that,NaN,helped,NaN,after the exam.
156,20,e,1,0,0,0,The principal knows,what,the counselor,helped,the student,after the exam.
157,20,f,1,1,1,0,The principal knows,what,NaN,helped,the student,after the exam.
158,20,g,1,1,0,1,The principal knows,what,the counselor,helped,NaN,after the exam.


In [ ]:
# columns: set_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_presence (0 = non-island, 1 = island), island_type, surprisal
sentence_df = pd.read_csv('test_construction.csv')
sentence_df['wh_licensor'] = sentence_df['wh_licensor'].astype('category')
sentence_df['gap'] = sentence_df['gap'].astype('category')
sentence_df['island_presence'] = sentence_df['island_presence'].astype('category')
sentence_df['island_type'] = sentence_df['island_type'].astype('category')

# list to store results
interaction_results = []

# a region filtering step is probably needed here
region_df = sentence_df[sentence_df['region'] == '']

def mixed_effects_linear_regression(df, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    model = smf.mixedlm(
        "surprisal ~ wh_licensor * gap * island_type",
        df,
        groups = df["set_id"],
        re_formula = "~wh_licensor + gap + island_type"
)

    result = model.fit()
    interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)
    
    print(f"\n=== {label.upper()} ===")
    print(result.summary())
    
    return interaction_coef


interaction = mixed_effects_linear_regression(region_df, "construction_type") # label name to be changed according to construction type